# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [1]:
from os import getcwd
from ticktack.fitting import SingleFitter
from ticktack import load_presaved_model
from jax.numpy import array, pi, mean, arange, full
from csv import writer

In [2]:
carbon_box_model = load_presaved_model(     # Generating the CarbobBoxModel using ticktack
    "Guttler14",                            # Selecting an arbitrary model 
    production_rate_units="atoms/cm^2/s"    # Units of the model
)

bayesian_model = SingleFitter(carbon_box_model)   # Fitting a model 
bayesian_model.prepare_function(model="simple_sinusoid") # Generating the simple sin model
bayesian_model.load_data(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv")   

mcmc_samples = bayesian_model.MarkovChainSampler(
    array([775., 1./12, pi/2., 81./12]),    # Initial guess of the parameters
    likelihood=bayesian_model.log_joint_simple_sinusoid,    
    args=(array([770., 0., -pi, 0.]), array([780., 5., pi, 15.])) # Upper and lower bounds
)   

INFO[2022-01-14 12:56:46,259]: Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
INFO[2022-01-14 12:56:46,260]: Unable to initialize backend 'gpu': NOT_FOUND: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO[2022-01-14 12:56:46,261]: Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.
WARNING[2022-01-14 12:56:46,261]: No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Running burn-in...


100%|██████████| 500/500 [00:39<00:00, 12.79it/s]


Running production...


100%|██████████| 1000/1000 [01:24<00:00, 11.82it/s]


In [9]:
bayesian_model.annual = arange(750, 800)    # 50 year window with the event in the middle
bayesian_model.mask = full((50,), True, dtype=bool) # Include all 50 years

event_parameters = mean(mcmc_samples, axis=0)   # The mcmc fitted parameters
event_parameters = array([*event_parameters[1::2]])  # Removing the event 